In [ ]:
#after training the model, we can use this file to test the model. RUn this notebook in A100 GPU machine (NC24adsA100 compute instance) with 

In [ ]:
# import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
import time
from azure.ai.ml import MLClient, Input
from azure.ai.ml.dsl import pipeline
from azure.ai.ml import load_component
credential = DefaultAzureCredential()
subscription_id = "" # your subscription id
resource_group = "" #your resource group
workspace = "" #your workspace name
workspace_ml_client = MLClient(credential, subscription_id, resource_group, workspace)

In [ ]:
model_name = "llama2_13b_fine_tuned"
model_path="./"
workspace_ml_client.models.download(model_name, version="2",download_path=model_path)
#after this step, remove the redundant parent folder name "llama2_13b_fine_tuned" so that the downloaded folder only has one 

In [ ]:
import mlflow
import pandas as pd
model = mlflow.pyfunc.load_model(model_name)
prompt = """
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Generate a chessboard with the given size and with pieces at the specified positions.
### Input:
Size: 8x8 \nPiece positions:\nWhite Rook at H8\nWhite Pawn at B3\nBlack Pawn at A3\nBlack Rook at H2

"""
model_input= {"text":[prompt], "max_length":100}
model.predict(model_input)
